In [1]:
import pandas as pd
import numpy as np
import ast
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('gsearch_jobs.csv')

In [3]:
df = df.drop(df.columns[[1,2]], axis=1)

In [4]:
df

,Unnamed: 0,company_name,location,via,description,extensions,job_id,thumbnail,posted_at,schedule_type,...,commute_time,salary_pay,salary_rate,salary_avg,salary_min,salary_max,salary_hourly,salary_yearly,salary_standardized,description_tokens
0,0,Cisco Meraki,Anywhere,via LinkedIn,"As the leader in cloud-managed IT, Cisco Merak...","['22 hours ago', 'Work from home', 'Full-time'...",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,https://encrypted-tbn0.gstatic.com/images?q=tb...,22 hours ago,Full-time,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['tableau', 'sql', 'aws']"
1,1,Midwest Staffing,Anywhere,via LinkedIn,As a Senior Business Analyst you will contribu...,"['53 minutes ago', 'Work from home', 'Full-tim...",eyJqb2JfdGl0bGUiOiJFbnRyeSBMZXZlbCAtIEJ1c2luZX...,https://encrypted-tbn0.gstatic.com/images?q=tb...,53 minutes ago,Full-time,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
2,2,"Amyx, Inc.",United States,via Indeed,Overview:\n\nAmyx is seeking to hire a Data An...,"['3 hours ago', 'Full-time']",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QvUmVzZWFyY2...,NaN,3 hours ago,Full-time,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['sql', 'azure', 'excel']"
3,3,Upwork,Anywhere,via Upwork,I am looking for someone to help me build an A...,"['23 hours ago', 'Work from home', 'Contractor']",eyJqb2JfdGl0bGUiOiJEYXRhIGFuYWx5c3QvQUkgZXhwZX...,NaN,23 hours ago,Contractor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['excel']
4,4,Bank Street College of Education,United States,via Higher Education Recruitment Consortium (H...,Position Vacancy – Data Analyst to support the...,"['20 hours ago', 'Full-time']",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,NaN,20 hours ago,Full-time,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['excel', 'powerpoint']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37175,37175,EDWARD JONES,"Houstonia, MO",via My ArkLaMiss Jobs,"At Edward Jones, we help clients achieve their...","['23 hours ago', '76,798–130,764 a year', 'Ful...",eyJqb2JfdGl0bGUiOiJNYXJrZXRpbmcgRGF0YSBcdTAwMj...,NaN,23 hours ago,Full-time,...,NaN,76798–130764,a year,103781.0,76798.0,130764.0,NaN,103781.0,103781.0,"['power_bi', 'excel', 'r', 'python', 'tableau'..."
37176,37176,EDWARD JONES,"Marshfield, MO",via My ArkLaMiss Jobs,"At Edward Jones, we help clients achieve their...","['23 hours ago', '106,916–182,047 a year', 'Fu...",eyJqb2JfdGl0bGUiOiJMZWFkLURhdGEgQW5hbHlzdCIsIm...,NaN,23 hours ago,Full-time,...,NaN,106916–182047,a year,144481.5,106916.0,182047.0,NaN,144481.5,144481.5,[]
37177,37177,EDWARD JONES,"High Point, MO",via My ArkLaMiss Jobs,"At Edward Jones, we help clients achieve their...","['23 hours ago', '106,916–182,047 a year', 'Fu...",eyJqb2JfdGl0bGUiOiJMZWFkLURhdGEgQW5hbHlzdCIsIm...,NaN,23 hours ago,Full-time,...,NaN,106916–182047,a year,144481.5,106916.0,182047.0,NaN,144481.5,144481.5,[]
37178,37178,EDWARD JONES,"Calhoun, MO",via My ArkLaMiss Jobs,"At Edward Jones, we help clients achieve their...","['23 hours ago', '106,916–182,047 a year', 'Fu...",eyJqb2JfdGl0bGUiOiJMZWFkLURhdGEgQW5hbHlzdCIsIm...,NaN,23 hours ago,Full-time,...,NaN,106916–182047,a year,144481.5,106916.0,182047.0,NaN,144481.5,144481.5,[]


In [5]:
df_clean = df.drop(columns = ['salary','via', 'description','extensions', 'job_id', 'thumbnail', 'posted_at', 'search_term', 'date_time', 'commute_time']).copy()

In [6]:
salary_columns = ['salary_pay', 'salary_rate', 'salary_avg', 'salary_min', 'salary_max', 'salary_hourly', 'salary_yearly']
df_clean.dropna(subset = salary_columns, how='all', inplace=True)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6430 entries, 5 to 37179
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           6430 non-null   int64  
 1   company_name         6430 non-null   object 
 2   location             6424 non-null   object 
 3   schedule_type        6424 non-null   object 
 4   work_from_home       4471 non-null   object 
 5   search_location      6430 non-null   object 
 6   salary_pay           6430 non-null   object 
 7   salary_rate          6430 non-null   object 
 8   salary_avg           6430 non-null   float64
 9   salary_min           6064 non-null   float64
 10  salary_max           6064 non-null   float64
 11  salary_hourly        4253 non-null   float64
 12  salary_yearly        2164 non-null   float64
 13  salary_standardized  6430 non-null   float64
 14  description_tokens   6430 non-null   object 
dtypes: float64(6), int64(1), object(8)
memory 

In [7]:
df_clean['schedule_type'].unique()

array(['Full-time', 'Contractor', 'Part-time', nan,
       'Contractor and Temp work', 'Full-time and Part-time',
       'Full-time, Contractor, and Temp work', 'Internship',
       'Full-time, Part-time, and Internship', 'Part-time and Contractor',
       'Full-time and Internship', 'Full-time, Part-time, and Per diem',
       'Temp work', 'Full-time and Temp work', 'Full-time and Contractor',
       'Part-time, Contractor, and Temp work',
       'Full-time, Part-time, and Contractor'], dtype=object)

In [8]:
df_clean['schedule_type'] = np.where(df_clean['schedule_type'].str.contains('Full-time', case = False, na=False), 'Full-time', df_clean['schedule_type'])
df_clean['schedule_type'] = np.where(df_clean['schedule_type'].str.contains('Part-time', case = False, na=False), 'Part-time', df_clean['schedule_type'])
df_clean['schedule_type'] = np.where(df_clean['schedule_type'].str.contains('Contractor', case=False, na=False), 'Contractor', df_clean['schedule_type'])
df_clean['schedule_type'] = np.where(df_clean['schedule_type'].isna(), 'Full-time', df_clean['schedule_type'])
df_clean['work_from_home'] = np.where(df_clean['work_from_home'].isna() , 'False', df_clean['work_from_home'])
df_clean['schedule_type'].unique()

array(['Full-time', 'Contractor', 'Part-time', 'Internship', 'Temp work'],
      dtype=object)

In [9]:
df_clean['work_from_home'].unique()

array([True, 'False'], dtype=object)

In [10]:
df_clean['salary_rate'].unique()

array(['a year', 'an hour', 'a month'], dtype=object)

In [11]:
for k, row in df_clean.iterrows():
    if row['salary_rate'] == 'a year':
        df_clean.at[k,'yearly_min'] = row['salary_min']
        df_clean.at[k,'yearly_max'] = row['salary_max']
    ### Assuming someone works 40 hours and 52 weeks
    if row['salary_rate'] == 'an hour':
        df_clean.at[k,'yearly_min'] = row['salary_min'] * 40 * 52
        df_clean.at[k,'yearly_max'] = row['salary_max'] * 40 * 52
    ### Assuming someone works the full year
    if row['salary_rate'] == 'a month':
        df_clean.at[k,'yearly_min'] = row['salary_min'] * 12
        df_clean.at[k,'yearly_max'] = row['salary_max'] * 12
df_clean

,Unnamed: 0,company_name,location,schedule_type,work_from_home,search_location,salary_pay,salary_rate,salary_avg,salary_min,salary_max,salary_hourly,salary_yearly,salary_standardized,description_tokens,yearly_min,yearly_max
5,5,Marathon TS,Anywhere,Full-time,True,United States,80K–90K,a year,85000.0,80000.0,90000.0,NaN,85000.0,85000.0,[],80000.0,90000.0
21,21,Upwork,Anywhere,Contractor,True,United States,20–40,an hour,30.0,20.0,40.0,30.0,NaN,62400.0,[],41600.0,83200.0
25,25,Insight Global,Anywhere,Full-time,True,United States,40–65,an hour,52.5,40.0,65.0,52.5,NaN,109200.0,"['pandas', 'numpy', 'matplotlib', 'pyspark', '...",83200.0,135200.0
52,52,Upwork,Anywhere,Contractor,True,United States,75–125,an hour,100.0,75.0,125.0,100.0,NaN,208000.0,[],156000.0,260000.0
65,65,Upwork,Anywhere,Contractor,True,United States,75–100,an hour,87.5,75.0,100.0,87.5,NaN,182000.0,[],156000.0,208000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37175,37175,EDWARD JONES,"Houstonia, MO",Full-time,False,United States,76798–130764,a year,103781.0,76798.0,130764.0,NaN,103781.0,103781.0,"['power_bi', 'excel', 'r', 'python', 'tableau'...",76798.0,130764.0
37176,37176,EDWARD JONES,"Marshfield, MO",Full-time,False,United States,106916–182047,a year,144481.5,106916.0,182047.0,NaN,144481.5,144481.5,[],106916.0,182047.0
37177,37177,EDWARD JONES,"High Point, MO",Full-time,False,United States,106916–182047,a year,144481.5,106916.0,182047.0,NaN,144481.5,144481.5,[],106916.0,182047.0
37178,37178,EDWARD JONES,"Calhoun, MO",Full-time,False,United States,106916–182047,a year,144481.5,106916.0,182047.0,NaN,144481.5,144481.5,[],106916.0,182047.0


In [12]:
df_clean.drop(columns = ['Unnamed: 0','salary_pay', 'salary_avg', 'salary_min', 'salary_max', 'salary_hourly', 'salary_yearly', 'salary_standardized',], inplace=True)
df_clean

,company_name,location,schedule_type,work_from_home,search_location,salary_rate,description_tokens,yearly_min,yearly_max
5,Marathon TS,Anywhere,Full-time,True,United States,a year,[],80000.0,90000.0
21,Upwork,Anywhere,Contractor,True,United States,an hour,[],41600.0,83200.0
25,Insight Global,Anywhere,Full-time,True,United States,an hour,"['pandas', 'numpy', 'matplotlib', 'pyspark', '...",83200.0,135200.0
52,Upwork,Anywhere,Contractor,True,United States,an hour,[],156000.0,260000.0
65,Upwork,Anywhere,Contractor,True,United States,an hour,[],156000.0,208000.0
...,...,...,...,...,...,...,...,...,...
37175,EDWARD JONES,"Houstonia, MO",Full-time,False,United States,a year,"['power_bi', 'excel', 'r', 'python', 'tableau'...",76798.0,130764.0
37176,EDWARD JONES,"Marshfield, MO",Full-time,False,United States,a year,[],106916.0,182047.0
37177,EDWARD JONES,"High Point, MO",Full-time,False,United States,a year,[],106916.0,182047.0
37178,EDWARD JONES,"Calhoun, MO",Full-time,False,United States,a year,[],106916.0,182047.0


In [13]:
df_clean['description_tokens'].value_counts()

description_tokens
[]                                                                                                                                                     1943
['excel']                                                                                                                                               310
['jira', 'microstrategy', 'tableau', 'sql']                                                                                                             257
['sql']                                                                                                                                                 160
['swift', 'excel', 'microstrategy', 'tableau', 'sql', 'powerpoint']                                                                                      92
                                                                                                                                                       ... 
['pandas', 'numpy', 'sql', 'spark', 'python']

In [14]:
for k, row in df_clean.iterrows():
    if row['description_tokens'] == '[]':
        df_clean.at[k, 'description_tokens'] = None

In [15]:
df_clean

,company_name,location,schedule_type,work_from_home,search_location,salary_rate,description_tokens,yearly_min,yearly_max
5,Marathon TS,Anywhere,Full-time,True,United States,a year,None,80000.0,90000.0
21,Upwork,Anywhere,Contractor,True,United States,an hour,None,41600.0,83200.0
25,Insight Global,Anywhere,Full-time,True,United States,an hour,"['pandas', 'numpy', 'matplotlib', 'pyspark', '...",83200.0,135200.0
52,Upwork,Anywhere,Contractor,True,United States,an hour,None,156000.0,260000.0
65,Upwork,Anywhere,Contractor,True,United States,an hour,None,156000.0,208000.0
...,...,...,...,...,...,...,...,...,...
37175,EDWARD JONES,"Houstonia, MO",Full-time,False,United States,a year,"['power_bi', 'excel', 'r', 'python', 'tableau'...",76798.0,130764.0
37176,EDWARD JONES,"Marshfield, MO",Full-time,False,United States,a year,None,106916.0,182047.0
37177,EDWARD JONES,"High Point, MO",Full-time,False,United States,a year,None,106916.0,182047.0
37178,EDWARD JONES,"Calhoun, MO",Full-time,False,United States,a year,None,106916.0,182047.0


In [16]:
## Creating a DataFrame with all unique skills required
df_skill = pd.DataFrame()
def safe_literal_eval(x):
    try:
        if x is not None:
            return ast.literal_eval(x)
        else:
            return []
    except (SyntaxError, ValueError):
        return []
df_clean['description_tokens'] = df_clean['description_tokens'].apply(safe_literal_eval)
unique_skills = set()
for skill_list in df_clean['description_tokens']:
    for skill in skill_list:
        unique_skills.add(skill)
for skill in unique_skills:
    counts = df_clean['description_tokens'].apply(lambda x:
                                                  x.count(skill))
    df_skill[skill] = counts
df_clean.drop(columns = ['description_tokens'], inplace=True)







In [17]:
df_skill

,nosql,sap,tensorflow,js,shell,power_bi,jupyter,github,java,ssis,...,t-sql,julia,scala,excel,datarobot,mxnet,unix,sharepoint,node,terminal
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
65,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37175,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
37176,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37177,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37178,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
df_clean



,company_name,location,schedule_type,work_from_home,search_location,salary_rate,yearly_min,yearly_max
5,Marathon TS,Anywhere,Full-time,True,United States,a year,80000.0,90000.0
21,Upwork,Anywhere,Contractor,True,United States,an hour,41600.0,83200.0
25,Insight Global,Anywhere,Full-time,True,United States,an hour,83200.0,135200.0
52,Upwork,Anywhere,Contractor,True,United States,an hour,156000.0,260000.0
65,Upwork,Anywhere,Contractor,True,United States,an hour,156000.0,208000.0
...,...,...,...,...,...,...,...,...
37175,EDWARD JONES,"Houstonia, MO",Full-time,False,United States,a year,76798.0,130764.0
37176,EDWARD JONES,"Marshfield, MO",Full-time,False,United States,a year,106916.0,182047.0
37177,EDWARD JONES,"High Point, MO",Full-time,False,United States,a year,106916.0,182047.0
37178,EDWARD JONES,"Calhoun, MO",Full-time,False,United States,a year,106916.0,182047.0


df_skill.to_csv('Skills.csv', sep=',', index=False, encoding='utf-8')

df_clean.to_csv('Cleaned.csv', sep=',', index=False, encoding='utf-8')